In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import datetime

import ta, pickle, json, time

In [2]:
data = pd.read_csv("1-Data creation/data_creation.csv")

In [41]:
df = pd.read_csv("data 1 year/BTCUSDT.csv").drop(
    ["bear", "bull", "ema20", "ema50", "ema100"], axis=1
)
df

,timestamp,open,high,low,close,volume
0,2020-07-11 22:17:00,9226.00,9226.00,9223.30,9224.20,8.967077
1,2020-07-11 22:18:00,9224.21,9225.00,9223.02,9223.80,11.122493
2,2020-07-11 22:19:00,9223.78,9226.46,9223.05,9225.99,14.178658
3,2020-07-11 22:20:00,9225.99,9232.06,9225.99,9231.96,53.251515
4,2020-07-11 22:21:00,9231.96,9235.00,9231.96,9235.00,22.051639
...,...,...,...,...,...,...
524643,2021-07-11 22:13:00,34464.07,34499.00,34457.49,34462.99,57.726496
524644,2021-07-11 22:14:00,34462.98,34478.60,34448.99,34451.15,33.589879
524645,2021-07-11 22:15:00,34451.14,34451.98,34403.22,34424.54,115.045592
524646,2021-07-11 22:16:00,34424.54,34424.54,34381.83,34381.84,61.389608


In [39]:
df = pd.read_csv("1-Data creation/0-price.csv")
df

,close,high,low,volume
0,0.000000,0.719644,-0.078689,0.788765
1,0.099933,0.418929,0.088712,0.781644
2,0.198866,0.558055,-0.420389,0.867857
3,0.295807,1.216390,-0.318883,0.781836
4,0.389787,0.576759,0.100879,0.019903
...,...,...,...,...
995,-0.802435,-0.752250,-1.276953,0.855017
996,-0.738785,-0.572666,-1.653521,0.886360
997,-0.667737,0.291070,-1.124593,0.181863
998,-0.590005,-0.292304,-0.772944,0.745340


In [40]:
data = df.copy()
data

,close,high,low,volume
0,0.000000,0.719644,-0.078689,0.788765
1,0.099933,0.418929,0.088712,0.781644
2,0.198866,0.558055,-0.420389,0.867857
3,0.295807,1.216390,-0.318883,0.781836
4,0.389787,0.576759,0.100879,0.019903
...,...,...,...,...
995,-0.802435,-0.752250,-1.276953,0.855017
996,-0.738785,-0.572666,-1.653521,0.886360
997,-0.667737,0.291070,-1.124593,0.181863
998,-0.590005,-0.292304,-0.772944,0.745340


In [41]:
# SMA, EMA, VWAP 6

for name, f in [("sma", ta.trend.sma_indicator), ("ema", ta.trend.ema_indicator)]:
    for x in [20, 50, 100]:
        df[f"{name}_{x}"] = f(close=df.close, window=x, fillna=True)
for x in [20, 50, 100]:
    df[f"vwap_{x}"] = ta.volume.volume_weighted_average_price(
        close=df.close, high=df.high, low=df.low, fillna=True, volume=df.volume, window=x
    )
for name in ["sma", "ema", "vwap"]:
    for fast, slow in [(20, 50), (20, 100), (50, 100)]:
        data[f"{name}_{fast}_{slow}"] = (
            df[f"{name}_{fast}"] > df[f"{name}_{slow}"]
        ).replace({False: 0, True: 1})

In [42]:
# Awesome 1
data[f"awesome_oscillator"] = (
    ta.momentum.awesome_oscillator(high=df.high, low=df.low, fillna=True) > 0
).replace({False: 0, True: 1})

In [43]:
# KAMA 1 
data[f"awesome_oscillator"] = (
    ta.momentum.kama(close=df.close, fillna=True) > df.close
).replace({False: 0, True: 1})

In [44]:
# PPO 2
ppo = ta.momentum.PercentagePriceOscillator(close=df.close, fillna=True)
data[f"ppo"] = (
    ppo.ppo() > 0
).replace({False: 0, True: 1})
data[f"ppo_hist"] = (
    ppo.ppo_hist() > 0
).replace({False: 0, True: 1})

In [45]:
# PVO 2
pvo = ta.momentum.PercentageVolumeOscillator(volume=df.volume, fillna=True)
data[f"pvo"] = (
    pvo.pvo() > 0
).replace({False: 0, True: 1})
data[f"pvo_hist"] = (
    pvo.pvo_hist() > 0
).replace({False: 0, True: 1})

In [46]:
# RSI 3
for name in ["close", "high", "low"]:
    data[f"rsi_{name}"] = ta.momentum.rsi(close=df[name], fillna=True) / 100

In [47]:
# S_RSI 3
s_rsi = ta.momentum.StochRSIIndicator(close=df.close, fillna=True)
data["srsi"] = s_rsi.stochrsi()
data["srsi_d"] = s_rsi.stochrsi_d()
data["srsi_k"] = s_rsi.stochrsi_k()

In [48]:
# S_RSI 2
s_osc = ta.momentum.StochasticOscillator(close=df.close, high=df.high, low=df.low, fillna=True)
data["stoch"] = s_osc.stoch() / 100
data["stoch_signal"] = s_osc.stoch_signal() / 100

In [49]:
# TSI 1
data[f"tsi"] = (ta.momentum.tsi(close=df.close, fillna=True) + 100) / 200

In [50]:
# Ultim_O 1
data[f"ultimate_o"] = ta.momentum.ultimate_oscillator(
    close=df.close, high=df.high, low=df.low, fillna=True
) / 100

In [51]:
# will 1
data[f"will"] = (ta.momentum.williams_r(
    close=df.close, high=df.high, low=df.low, fillna=True
) + 100) / 100

In [52]:
# CMF 1
data[f"cmf"] = (ta.volume.chaikin_money_flow(
    close=df.close, high=df.high, low=df.low, fillna=True, volume=df.volume
) + 1 ) / 2

In [53]:
# EM 2
ease_movement = ta.volume.EaseOfMovementIndicator(
    high=df.high, low=df.low, fillna=True, volume=df.volume
)
data[f"em"] = (
    ease_movement.ease_of_movement() > 0
).replace({False: 0, True: 1})
data[f"sem"] = (
    ease_movement.sma_ease_of_movement() > 0
).replace({False: 0, True: 1})

In [54]:
# MFI 1
data[f"mfi"] = ta.volume.money_flow_index(
    close=df.close, high=df.high, low=df.low, fillna=True, volume=df.volume
) / 100

In [55]:
# BB 3
bb = ta.volatility.BollingerBands(close=df.close)
data[f"bbh"] = bb.bollinger_hband_indicator()
data[f"bbl"] = bb.bollinger_lband_indicator()
data[f"bbm"] = (
    bb.bollinger_mavg() > df.close
).replace({False: 0, True: 1})

In [56]:
# DC 3
dc = ta.volatility.DonchianChannel(close=df.close, high=df.high, low=df.low)
data[f"dch"] = (
    dc.donchian_channel_hband() < df.close
).replace({False: 0, True: 1})
data[f"dcl"] = (
    dc.donchian_channel_lband() > df.close
).replace({False: 0, True: 1})
data[f"dcm"] = (
    dc.donchian_channel_mband() > df.close
).replace({False: 0, True: 1})

In [57]:
# KC 3
kc = ta.volatility.KeltnerChannel(close=df.close, high=df.high, low=df.low)
data[f"kch"] = kc.keltner_channel_hband_indicator()
data[f"kcl"] = kc.keltner_channel_lband_indicator()
data[f"kcm"] = (
    kc.keltner_channel_mband() > df.close
).replace({False: 0, True: 1})

In [58]:
# ADX 3
adx = ta.trend.ADXIndicator(close=df.close, high=df.high, low=df.low, fillna=True) 
data[f"adx"] = adx.adx() / 100
data[f"adx_neg"] = adx.adx_neg() / 100
data[f"adx_pos"] = adx.adx_pos() / 100

/home/tttienthinh/Documents/Programmation/Scientist/venv/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/tttienthinh/Documents/Programmation/Scientist/venv/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [59]:
# Aroon 3
aroon = ta.trend.AroonIndicator(close=df.close, fillna=True) 
data[f"aroon_d"] = aroon.aroon_down() / 100
data[f"aroon_u"] = aroon.aroon_up() / 100
data[f"aroon_i"] = (aroon.aroon_indicator() + 100) / 200

In [60]:
# CCI 1
data[f"cci"] = (
    ta.trend.cci(close=df.close, high=df.high, low=df.low, fillna=True) > 0
).replace({False: 0, True: 1})

In [61]:
# DPO 1
data[f"dpo"] = (
    ta.trend.dpo(close=df.close, fillna=True) > 0
).replace({False: 0, True: 1})

In [62]:
# KST 3
kst = ta.trend.KSTIndicator(close=df.close, fillna=True)
data[f"kst"] = (
    kst.kst() > 0
).replace({False: 0, True: 1})
data[f"kst_diff"] = (
    kst.kst_diff() > 0
).replace({False: 0, True: 1})
data[f"kst_sig"] = (
    kst.kst_sig() > 0
).replace({False: 0, True: 1})

In [63]:
# MACD 3
macd = ta.trend.MACD(close=df.close, fillna=True)
data[f"macd"] = (
    macd.macd() > 0
).replace({False: 0, True: 1})
data[f"macd_d"] = (
    macd.macd_diff() > 0
).replace({False: 0, True: 1})
data[f"macd_s"] = (
    macd.macd_signal() > 0
).replace({False: 0, True: 1})

In [64]:
# PSAR
psar = ta.trend.PSARIndicator(close=df.close, high=df.high, low=df.low, fillna=True)
data["psar_d"] = psar.psar_down_indicator()
data["psar_u"] = psar.psar_up_indicator()

In [65]:
# STC 1
data["stc"] = ta.trend.stc(close=df.close, fillna=True) / 100

In [66]:
# trix 1
data["trix"] = (
    ta.trend.trix(close=df.close, fillna=True) > 0
).replace({False: 0, True: 1})

In [67]:
# vortex 1
vortex = ta.trend.VortexIndicator(close=df.close, high=df.high, low=df.low, fillna=True)
data["vortex"] = (
    vortex.vortex_indicator_diff() > 0
).replace({False: 0, True: 1})

In [68]:
data

,close,high,low,volume,sma_20_50,sma_20_100,sma_50_100,ema_20_50,ema_20_100,ema_50_100,...,kst_diff,kst_sig,macd,macd_d,macd_s,psar_d,psar_u,stc,trix,vortex
0,0.000000,0.719644,-0.078689,0.788765,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.000000,0,0
1,0.099933,0.418929,0.088712,0.781644,0,0,0,1,1,1,...,1,1,1,1,1,0.0,0.0,0.000000,0,0
2,0.198866,0.558055,-0.420389,0.867857,0,0,0,1,1,1,...,1,1,1,1,1,1.0,0.0,0.000000,1,0
3,0.295807,1.216390,-0.318883,0.781836,0,0,0,1,1,1,...,1,1,1,1,1,0.0,1.0,0.000000,1,1
4,0.389787,0.576759,0.100879,0.019903,0,0,0,1,1,1,...,1,1,1,1,1,0.0,0.0,0.000000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.802435,-0.752250,-1.276953,0.855017,0,0,1,0,0,0,...,1,1,0,1,0,0.0,0.0,0.750000,1,0
996,-0.738785,-0.572666,-1.653521,0.886360,0,0,1,0,0,0,...,1,1,0,1,0,1.0,0.0,0.875000,1,0
997,-0.667737,0.291070,-1.124593,0.181863,0,0,1,0,0,0,...,1,1,0,1,0,0.0,1.0,0.937500,1,1
998,-0.590005,-0.292304,-0.772944,0.745340,0,0,1,0,0,0,...,0,1,0,1,0,0.0,0.0,0.968750,1,1


In [69]:
data.to_csv("1-Data creation/0-data.csv.csv", index=False)